# Vorlesung: Pandas Dataframes

Pandas: http://www.data-analysis-in-python.org/3_pandas.html
      : https://bitbucket.org/hrojas/learn-pandas

In [30]:
# <api>
import json
import pandas as pd
import re
import numpy as np
import requests

## Construct Dataframe from full Poleis data

In [31]:
# <api>
PoleisDataOnline2 = requests.get('http://repository.edition-topoi.org/MISC/ReposMISC/MISC00005/secondVersion.json')
PoleisRawData2 = PoleisDataOnline2.json()
PoleisRawData2.keys()

dict_keys(['The Propontic Coast of Asia Minor', 'Arkadia', 'Lykia', 'Ionia', 'Epeiros', 'The South Coast of Asia Minor (Pamphylia Kilikia)', 'Cyprus', 'Achaia', 'The Adriatic', 'The Aegean', 'Makedonia', 'Italia and Kampania', 'Elis', 'West Lokris', 'East Lokris', 'Boiotia', 'Thrace from Axios to Strymon', 'Lakedaimon', 'Rhodos', 'Thracian Chersonesos', 'Aitolia', 'Lesbos', 'Aiolis and South-western Mysia', 'Thrace from Nestos to Hebros', 'The Saronic Gulf', 'The Black Sea Area', 'Propontic Thrace', 'Thessalia and Adjacent Regions', 'Doris', 'Spain and France (including Corsica)', 'Messenia', 'Karia', 'Megaris, Korinthia, Sikyonia', 'Akarnania and Adjacent Areas', 'Thrace from Strymon to Nestos', 'Triphylia', 'Phokis', 'Sikelia', 'Troas', 'Attika', 'Euboia', 'Argolis', 'Inland Thrace', 'Crete'])

In [32]:
# <api>
# Read Json into a normalized form, yields ~500 columns with region.city keys
dfPoleisGesamt = pd.io.json.json_normalize(PoleisRawData2)

# rotate and rename dataframe
dfPoleisGesamt= dfPoleisGesamt.transpose()
dfPoleisGesamt.columns=['Beschreibung']
dfPoleisGesamt.head(4)

# reset to new index, return old index as column 'index'
dfPoleisGesamt= dfPoleisGesamt.reset_index()
dfPoleisGesamt.head()

# split entries in column 'index' into region and city part
dfPoleisGesamt['indexSplit'] = dfPoleisGesamt['index'].str.split('.')

# generate new columns out of split index
dfPoleisGesamt['region'] = dfPoleisGesamt['indexSplit'].apply(lambda raw: raw[0])
dfPoleisGesamt['city'] = dfPoleisGesamt['indexSplit'].apply(lambda raw: raw[1])
dfPoleisGesamt.head()

# remove columns 'index' and 'indexSplit', since they contain redundant information
dfPoleisGesamt = dfPoleisGesamt.drop('index', 1)
dfPoleisGesamt = dfPoleisGesamt.drop('indexSplit', 1)
dfPoleisGesamt.head()

,Beschreibung,region,city
0,"Identifier: 233. , (Ascheieus) Unlocated. Typ...",Achaia,Ascheion
1,"Identifier: 235. , (Bourios) Map 58. Lat. 38...",Achaia,Boura
2,"Identifier: 236. , (Helikeus) Map 58. Lat. 3...",Achaia,Helike
3,"Identifier: 237. , (Keryneus) Map 58. Lat. 3...",Achaia,Keryneia
4,"Identifier: 238. , (Leontesios) Map 58.Lat.38...",Achaia,Leontion


In [33]:
# <api>
def ListePattern(string,pattern):
    x = re.findall(pattern,string)
    if x:
        return(x)

Example for geographical coordinates:

- (?<=Lat\.\s)  Group (?...) Passive (non-capturing) group
- ?<= Lookbehind assertion
- Lat\.\s   das string muster: "Lat. " mit "." und " " als escape
- \s?\d+\.\d+ : space[optional wegen ?]digit[1 oder 2 wegen +].[escaped]digit[1 oder 2]

Code::

    dfPoleisGesamt['Latitude'] = dfPoleisGesamt['Beschreibung'].apply(lambda raw: ListePattern(raw,"(?<=Lat\.\s)\d+\.\d+"))

## Add sources by regex

In [34]:
# <api>
pat = re.compile('([A-Z][a-z]{1,10}\. \d{1,3}\.\d{1,3}\.\d{1,3}|[A-Z][a-z]{1,15}\s{0,2}\(\s*\d{4}\)|[A-Z][a-z]{1,15}\s{0,2}\(\s*\d{1,2}\.\d{1,2}\.\d{1,2}\)|[A-Z][a-z]{1,15}\s{0,2}\s*\d{1,2}\.\d{1,2}\.\d{1,2}|[A-Z][a-z]{1,15}\.\s+[A-Z][a-z]{1,10}\.\s+[A-Za-z]{1,10}\.\s+\d{1,3}\.\d{1,3}|[A-Z][a-z]{1,15}\.\s+[A-Z][a-z]{1,10}\.\s+\d{1,3}\.\d{1,3}[A-Z][a-z]{1,10}.?\s+[¹²³]\d+\.\d+|[A-Z][a-z]{1,10}.?\s+[¹²³]\d+|[A-Z][a-z]{1,10}.?\s+\d+\s+\d+)')

dfPoleisGesamt['Quellen'] = dfPoleisGesamt['Beschreibung'].apply(lambda raw: ListePattern(raw,pat))

In [16]:
dfPoleisGesamt.head(10)

,Beschreibung,region,city,Quellen
0,"Identifier: 233. , (Ascheieus) Unlocated. Typ...",Achaia,Ascheion,None
1,"Identifier: 235. , (Bourios) Map 58. Lat. 38...",Achaia,Boura,"[Paus. 7.25.8, Strabo 8.7.5, Hall ( 1996), R..."
2,"Identifier: 236. , (Helikeus) Map 58. Lat. 3...",Achaia,Helike,"[Paus. 7.24.5, Strabo 8.7.2, Hall ( 1996), Di..."
3,"Identifier: 237. , (Keryneus) Map 58. Lat. 3...",Achaia,Keryneia,"[Paus. 7.25.5, Rizakis ( 1995), Paus. 7.25.5,..."
4,"Identifier: 238. , (Leontesios) Map 58.Lat.38...",Achaia,Leontion,"[Lauffer ( 1989), Polyb. 2.41.7, Strabo 8.7...."
5,"Identifier: 241. , (Olenios) Map 58. Lat. 38...",Achaia,Olenos,"[Paus. 7.18.1, Strabo 8.7.4, Rizakis ( 1995)..."
6,"Identifier: 244. , (Pharaieus) Map 58. Lat. ...",Achaia,Pharai,"[Paus. 7.22.1, Rizakis ( 1995), Polyb. 2.41.8,..."
7,"Identifier: hall, (Tritaieus) Map 58.Lat.37.5...",Achaia,Tritaia,"[Paus. 7.22.6, Rizakis ( 1995), Polyb. 2.41.8..."
8,"Identifier: 801. , (Adramytenos) Map 56. Lat...",Aiolis and South-western Mysia,Adramyttion,"[Hdt. 7.42.1, An. 7.8.8, Thuc. 5.1.1, Fossey ..."
9,"Identifier: 802. , (Aigaieus) Map 56. Lat. 3...",Aiolis and South-western Mysia,Aigai(ai),"[Hdt. 1.149.1, Strabo 13.3.5, Hell. 4.8.5, Pol..."


## Get city identifier by RegEx

Throughout the full text, cities are referenced by a running index. To make this information part of the dataframe, we extend it with an additional column.


In [35]:
def cityIDFinder(text):
    '''
    #1: Find all occurance of the string "Identifier" followed by a colon, a space and between one and four decimals.
    #2: If there is a result, do the following
    #3: Take the first result idList[0] (because the identifier is at the beginning of the text), 
        and split the string at the dot (to remove the dot at the end of the string). Then return the string from the 13. position.
        This ensures, that only a number is returned, since it removes the word identifier, the colon, and the space. 
    '''
    idList = re.findall("Identifier\: \d{1,4}\.", text) #1
    if idList: #2
        idCity = idList[0].split('.')[0][12:] #3
        return idCity

In [36]:
dfPoleisGesamt['city_id'] = dfPoleisGesamt['Beschreibung'].apply(lambda row: cityIDFinder(row))

## Collection of citations uding parenthesis detection

To collect citations in the text for one city, we first use a tokenizer from NLTK. This tokenizer collects all parenthesis and is much easier to use, that regular expressions.

The basic assumption for citations is: They are written in parenthesis, start with a capital letter, and contain at least one blank space (to separate the authors name from text pages, indices, or dates).


In [55]:
#pat = re.compile('([A-Z][a-z]{1,10}\. \d{1,3}\.\d{1,3}\.\d{1,3}|[A-Z][a-z]{1,15}\s{0,2}\(\s*\d{4}\)|[A-Z][a-z]{1,15}\s{0,2}\(\s*\d{1,2}\.\d{1,2}\.\d{1,2}\)|[A-Z][a-z]{1,15}\s{0,2}\s*\d{1,2}\.\d{1,2}\.\d{1,2}|[A-Z][a-z]{1,15}\.\s+[A-Z][a-z]{1,10}\.\s+[A-Za-z]{1,10}\.\s+\d{1,3}\.\d{1,3}|[A-Z][a-z]{1,15}\.\s+[A-Z][a-z]{1,10}\.\s+\d{1,3}\.\d{1,3}[A-Za-z]{1,10}.?\s+[¹²³]\d+\.\d+|[A-Za-z]{1,10}.?\s+[¹²³]\d+|[A-Za-z]{1,10}.?\s+\d+\s+\d+)')


In [19]:
#def citationFinder(text):
#   '''
#    #1: Generate a list of all capital letters
#    #2: Tokenize text to search for parenthesis, '( ... )' (this returns more accurate results, that using regular expression) 
#        The option strict=False is necessary to prevent errors, when the text contains only an unmatched opening or closing paranthesis.
#    #3: The basic assumptions for citations are: 
#        In parenthesis (first element is opening paranthesis), 
#        start with a capital letter (second element is a capital letter), 
#        and contain at least one blank space ' '
#    '''
#    import string
#    letters=[i for i in string.ascii_uppercase] #1
#    paranthesisTokenized = SExprTokenizer(strict=False).tokenize(text) #2
#    listCite = [x for x in paranthesisTokenized if x[0] == '(' and x[1] in letters and ' ' in x] #3
#    return listCite

In [29]:
#dfPoleisGesamt['sources'] = dfPoleisGesamt['Beschreibung'].apply(lambda row: citationFinder(row))

## Transformation of coordinates using RegEx

A simple regular expression is enough to find all coordinates in the text. The coordinates are transformed from degrees/minutes to decimal to enable plotting on a map with common projection.


In [21]:
def coordinateFinder(value,pattern):
    '''
    #1: General function for finding regular expression pattern in a text.
    #2: If patterns are found, do the following
    #3: Take the last five values of the first string returned
    #4: To convert angular in decimal coordinates: 
        Take the returned value, split it at the dot
        convert the first part into a floating number (e.g. 36.0), 
        and the second part into a integer number (e.g. 34) and divide it by 60. 
        The sum the two results to return a coordinate in decimal system
    '''
    x = re.findall(pattern, value)                                             #1
    if x:                                                                      #2  
        coord = x[0][-5:]                                                      #3 
        decCord = float(coord.split('.')[0]) +  int(coord.split('.')[-1])/60   #4
        return decCord

In [22]:
dfPoleisGesamt['latitude'] = dfPoleisGesamt["Beschreibung"].apply(coordinateFinder, pattern="Lat\.\s?\d+\.\d+")
dfPoleisGesamt['longitude'] = dfPoleisGesamt["Beschreibung"].apply(coordinateFinder, pattern="long\.\s*\d+\.\d+")

##  Proper nouns using POS tagger

To generate a list of all mentioned proper nouns for each city, we use TextBlob. TextBlob is a NLTK tool with parts-of-speech tagger. We are interessted in all parts that are 'NNP' and longer then 3 letters.

This takes some time to process for the full dataframe. Behaviour can be tested by uncommenting the cell below.


In [28]:
#def namesFinder(text):
#    '''
#    #1: Generate a blob out of the text 
#    #2: Generate a list of all POS Tags, that are labeld as NNP(S) (Proper noun, singular (or plural)), and which are longer than 3 letters 
#    '''
#    blobs = TextBlob(text)                                                                              #1
#    namesList = [x[0] for x in blobs.pos_tags if (x[1] == 'NNP') | (x[1] == 'NNPS') and len(x[0]) > 3]  #2
#    return namesList

In [27]:
# Uncomment to test routine. 
#namesFinder(dfPoleisGesamt['Beschreibung'].iloc[10])

In [62]:
########################################
# Careful: Takes some time to evalute! #
########################################

#dfPoleisGesamt['names'] = dfPoleisGesamt['Beschreibung'].apply(lambda row: namesFinder(row))

##  Cross links to other cities

Links to other cities are mentioned in the fulltext with reference to the index (e.g. '(no. 982)'). searching for these should give a link list.


In [23]:
def linksFinder(text):
    '''
    #1: Find all occurances of the string "(no. 1234)" with between one and four decimals
    #2: If we have a result
    #3: Generate a list, where every result:
        is split at the space, take the last part, and only up to the last letter (this removes the closing paranthesis) 
    #4: For all these results, convert the entries into an integer number
    '''
    x = re.findall("\(no\. \d{1,4}\)", text)               #1
    if x:                                                  #2
        links = [((z.split(' '))[-1])[:-1] for z in x]     #3
        linksInt = [int(x) for x in links]                 #4
        return linksInt

In [24]:
dfPoleisGesamt['linkedCities'] = dfPoleisGesamt['Beschreibung'].apply(lambda row: linksFinder(row))

## Display dataframe

In [26]:
dfPoleisGesamt.head(4)

,Beschreibung,region,city,Quellen,city_id,latitude,longitude,linkedCities
0,"Identifier: 233. , (Ascheieus) Unlocated. Typ...",Achaia,Ascheion,None,233,NaN,NaN,None
1,"Identifier: 235. , (Bourios) Map 58. Lat. 38...",Achaia,Boura,"[Paus. 7.25.8, Strabo 8.7.5, Hall ( 1996), R...",235,38.166667,22.250000,"[236, 235, 238, 251, 165, 148]"
2,"Identifier: 236. , (Helikeus) Map 58. Lat. 3...",Achaia,Helike,"[Paus. 7.24.5, Strabo 8.7.2, Hall ( 1996), Di...",236,38.250000,22.166667,"[231, 70]"
3,"Identifier: 237. , (Keryneus) Map 58. Lat. 3...",Achaia,Keryneia,"[Paus. 7.25.5, Rizakis ( 1995), Paus. 7.25.5,...",237,38.166667,22.166667,[353]
